Transaction validation

* Inputs are previously unspent
* Sum of inputs is greater than or equal to the outputs
* The SigScript unlocks the previous ScriptPubKey


In [3]:
# Example - is this transaction attempting to make money out of thin air?

from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf830\
3c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccf\
cf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8\
e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278\
afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88a\
c99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)  # <1>

/Users/ds/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True


ECDSA requires the public key P, the signature hash z, and the signature (r,s)

In [4]:
from ecc import S256Point, Signature
# SEC - standards for efficent cryptography - public key serialization
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e\
213bf016b278a')
# DER - distinguised encoding rules - for signatures
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031c\
cfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9\
c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


Getting the signature hash

* Can't hash the transaction, since the signature itself is embedded in the SigScript
* Transaction is modified before signing - a signature hash for each input is computed

The process:

1. Empty all ScriptSigs before checking (or for creating, probably has empty ScriptSigs when creating).
2. Replace the SigScript of the transaction with ScriptPubKey of the previous transaction output
3. Append the hash type encoded little-endian over 4 bytes

The hash256 of the modified transaction can be taken, and the big-endian integer is taken as z

In [5]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914\
bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896\
bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02\
e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288a\
c1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))


0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [6]:
# Putting it all together

from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e\
213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031c\
cfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9\
c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

In [7]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b"Nouvelle Methode complete de cornet a pistons et d'instruments a pistons par Alexandre Petit"))
private_key = PrivateKey(secret)
private_key.point.address(compressed=False, testnet=True)

'n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4'

In [15]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret2 = little_endian_to_int(hash256(b"Destination"))
private_key2 = PrivateKey(secret2)
private_key2.point.address(compressed=False, testnet=True)

'mhxabKdkyQvxm8yCciAk7hQiWyKs2PUWh2'

In [9]:
# Fund the address
!curl -X POST --data '{"address": "n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4", "amount": 0.02}' http://localhost:3000/faucet

{"txId":"233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e"}


On the command line

```
alias bcli='docker exec -it bitcoin bitcoin-cli'        
ds@dss-MacBook-Pro ~ % bcli -regtest getnewaddress                     
bcrt1q6zdhs72upcwpuu5c85e2k7ncum7sjjn5fhrjty
```

Current UTX0 set

```
bcli gettransaction 233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e
{
  "amount": -0.02000000,
  "fee": -0.00000147,
  "confirmations": 1,
  "blockhash": "3ea2bcd72d603715e94b2c2ff84663d7652669ea46ad555aecf22737b4acce52",
  "blockheight": 102,
  "blockindex": 1,
  "blocktime": 1694098554,
  "txid": "233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e",
  "wtxid": "1dcbe07b4024a1fcd0fe5a9e3df9d27f9e0bab5604baf582d2d8cfdf2d98d1a4",
  "walletconflicts": [
  ],
  "time": 1694098554,
  "timereceived": 1694098554,
  "bip125-replaceable": "no",
  "details": [
    {
      "address": "n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4",
      "category": "send",
      "amount": -0.02000000,
      "vout": 1,
      "fee": -0.00000147,
      "abandoned": false
    }
  ],
  "hex": "0200000000010195688d7e1337f698dd452434257c543a748219669111544b4503fe1d4605a9ec0000000000fdffffff02ed6ce729010000001976a9140fa738fd6297380aacb498c74ac038e9db83009588ac80841e00000000001976a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac02473044022001083413c63c76b93ff468f2ba693a042be14162902de83fcabbe79b32f33a340220632991f6226ebba1746c771f4ec82c835b7ae5735af6568374df0170f23c54c6012103efb2c883e93e2977c1341b8637d757ce47b970e02aa13acd65ca26f844de8d6400000000"
}
```


Can we send from the address controlled by our private key to the second address, receiving change back to our original address (not a recommended practive, but for simplicity)?

In [20]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Source address has 2,000,000 satoshis - send half, guess 2000 for the txn fee, rest is change back 998,000

prev_tx = bytes.fromhex('233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e')
prev_index = 1
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(998000)  
change_h160 = decode_base58('n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(1000000)  
target_h160 = decode_base58('mhxabKdkyQvxm8yCciAk7hQiWyKs2PUWh2')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)  
print(tx_obj)
print(tx_obj.serialize().hex())



tx: 44cb6244fc26ffffe067ec315e380a084e68198aa5a0a68dc474f7109be997b6
version: 1
tx_ins:
233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e:1
tx_outs:
998000:OP_DUP OP_HASH160 f69ff8808047eeddf564911ae7e0384b2cacf762 OP_EQUALVERIFY OP_CHECKSIG
1000000:OP_DUP OP_HASH160 1ac8f34f371bd9c05100f4004a821fb77469faf4 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
01000000014efd9b7e058916a92eefa3f43496f9889eef2bf26d7119673e6976c892d73b230100000000ffffffff02703a0f00000000001976a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac40420f00000000001976a9141ac8f34f371bd9c05100f4004a821fb77469faf488ac00000000


What does this look like if we decode it using the command line?

```
export RAW_TX=01000000014efd9b7e058916a92eefa3f43496f9889eef2bf26d7119673e6976c892d73b230100000000ffffffff02703a0f00000000001976a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac40420f00000000001976a9141ac8f34f371bd9c05100f4004a821fb77469faf488ac00000000

bcli decoderawtransaction $RAW_TX

{
  "txid": "44cb6244fc26ffffe067ec315e380a084e68198aa5a0a68dc474f7109be997b6",
  "hash": "44cb6244fc26ffffe067ec315e380a084e68198aa5a0a68dc474f7109be997b6",
  "version": 1,
  "size": 119,
  "vsize": 119,
  "weight": 476,
  "locktime": 0,
  "vin": [
    {
      "txid": "233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 0.00998000,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 f69ff8808047eeddf564911ae7e0384b2cacf762 OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4)#4redl4xr",
        "hex": "76a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac",
        "address": "n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4",
        "type": "pubkeyhash"
      }
    },
    {
      "value": 0.01000000,
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 1ac8f34f371bd9c05100f4004a821fb77469faf4 OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mhxabKdkyQvxm8yCciAk7hQiWyKs2PUWh2)#vc6aacjc",
        "hex": "76a9141ac8f34f371bd9c05100f4004a821fb77469faf488ac",
        "address": "mhxabKdkyQvxm8yCciAk7hQiWyKs2PUWh2",
        "type": "pubkeyhash"
      }
    }
  ]
}

```

In [21]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = tx_obj.sig_hash(0)  

der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')  
sec = private_key.point.sec()
script_sig = Script([sig, sec])  
tx_obj.tx_ins[0].script_sig = script_sig   
print(tx_obj.serialize().hex())



01000000014efd9b7e058916a92eefa3f43496f9889eef2bf26d7119673e6976c892d73b23010000006b483045022100e56124079fc642d15d7a47c1728a9c1f6a3effd0e95e64ee7b31c08cb3cae3be022010e392d15bc2a3acbf0379fe1e89c90b54e45357b229cb46b9177abf6e745b9f012102e00d1040a9afb99d351a5cf9294d029d828311d9558b0203015a4f07a6787c9effffffff02703a0f00000000001976a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac40420f00000000001976a9141ac8f34f371bd9c05100f4004a821fb77469faf488ac00000000


In [19]:
'0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a473044022043c180d763726ef41b1088619e70efa073ae65fc8f211f7eb1175e2db9654d97022075d18168ff944dd15914580e0656bba2ccbf6c321e2ffb853cbd5801b8c8277c012102e00d1040a9afb99d351a5cf9294d029d828311d9558b0203015a4f07a6787c9efeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600' == '0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a473044022043c180d763726ef41b1088619e70efa073ae65fc8f211f7eb1175e2db9654d97022075d18168ff944dd15914580e0656bba2ccbf6c321e2ffb853cbd5801b8c8277c012102e00d1040a9afb99d351a5cf9294d029d828311d9558b0203015a4f07a6787c9efeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600'

True

Send the transaction

```
export SIGNED=01000000014efd9b7e058916a92eefa3f43496f9889eef2bf26d7119673e6976c892d73b23010000006b483045022100e56124079fc642d15d7a47c1728a9c1f6a3effd0e95e64ee7b31c08cb3cae3be022010e392d15bc2a3acbf0379fe1e89c90b54e45357b229cb46b9177abf6e745b9f012102e00d1040a9afb99d351a5cf9294d029d828311d9558b0203015a4f07a6787c9effffffff02703a0f00000000001976a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac40420f00000000001976a9141ac8f34f371bd9c05100f4004a821fb77469faf488ac00000000


bcli -regtest sendrawtransaction $SIGNED
error code: -26
error message:
mandatory-script-verify-flag-failed (Script failed an OP_EQUALVERIFY operation)

```

Hmmm.... to troubleshoot decode the hex of the prev transaction

```
{
  "txid": "233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e",
  "hash": "1dcbe07b4024a1fcd0fe5a9e3df9d27f9e0bab5604baf582d2d8cfdf2d98d1a4",
  "version": 2,
  "size": 228,
  "vsize": 147,
  "weight": 585,
  "locktime": 0,
  "vin": [
    {
      "txid": "eca905461dfe03454b541191661982743a547c25342445dd98f637137e8d6895",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3044022001083413c63c76b93ff468f2ba693a042be14162902de83fcabbe79b32f33a340220632991f6226ebba1746c771f4ec82c835b7ae5735af6568374df0170f23c54c601",
        "03efb2c883e93e2977c1341b8637d757ce47b970e02aa13acd65ca26f844de8d64"
      ],
      "sequence": 4294967293
    }
  ],
  "vout": [
    {
      "value": 49.97999853,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 0fa738fd6297380aacb498c74ac038e9db830095 OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mgwikmBUvbYhFfRsvEYgTb6FZZsyGAFgzq)#pmlyzhm0",
        "hex": "76a9140fa738fd6297380aacb498c74ac038e9db83009588ac",
        "address": "mgwikmBUvbYhFfRsvEYgTb6FZZsyGAFgzq",
        "type": "pubkeyhash"
      }
    },
    {
      "value": 0.02000000,
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 f69ff8808047eeddf564911ae7e0384b2cacf762 OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4)#4redl4xr",
        "hex": "76a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac",
        "address": "n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4",
        "type": "pubkeyhash"
      }
    }
  ]
}
```

The previous script pub key for the output we want to spend is OP_DUP OP_HASH160 f69ff8808047eeddf564911ae7e0384b2cacf762 OP_EQUALVERIFY OP_CHECKSIG

This matches the 'send part', but what about the change part?